In [ ]:
import pyodbc
import pymysql
import time
from datetime import datetime
import threading
import signal
import sys

# ================== تنظیمات ==================
INTERVAL_SECONDS = 60 * 60  # هر ۶۰ دقیقه
RUNNING = True  # برای توقف با Ctrl+C

# ================== اتصالات ==================
def get_sql_connection():
    return pyodbc.connect(
        'DRIVER={SQL Server};'
        'SERVER=MKZ-DSAS\\DSAS;'
        'DATABASE=DSAS;'
        'UID=datadriven;'
        # توجه: اگر Trusted Connection دارید می‌تونید UID/PWD رو حذف کنید
        'PWD=5Rdx@4Rfv1355'
    )

def get_mysql_connection():
    return pymysql.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="",
        database="dsas",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

# ================== لیست جدید AssetIDها ==================
asset_ids = [8312, 8313, 8314]

# نگاشت فیلدهای جدول MySQL به AssetID
field_map = {
    8312: 'AssetID_8312',
    8313: 'AssetID_8313',
    8314: 'AssetID_8314'
}

# ================== تابع اصلی ==================
def run_task():
    print(f"\n{'='*70}")
    print(f"اجرای وظیفه: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"جدول هدف: main_table_mhi_main_trans_12")
    print(f"AssetIDهای مورد استفاده: {asset_ids}")
    print(f"{'='*70}")

    conn_sql = None
    conn_mysql = None
    cursor_sql = None

    try:
        # --- اتصال ---
        print("اتصال به SQL Server...")
        conn_sql = get_sql_connection()
        cursor_sql = conn_sql.cursor()

        print("اتصال به MySQL...")
        conn_mysql = get_mysql_connection()

        # --- خواندن TimeStamps موجود در جدول جدید ---
        print("خواندن TimeStampsهای موجود در MySQL...")
        existing_ts = set()
        with conn_mysql.cursor() as c:
            c.execute("SELECT TimeStamps FROM main_table_mhi_main_trans_12 WHERE unitID = 12")
            for row in c.fetchall():
                existing_ts.add(row['TimeStamps'])
        print(f"{len(existing_ts)} رکورد موجود در جدول MySQL.")

        # --- خواندن آخرین رکوردهای AssetID اصلی (8312) ---
        print("خواندن آخرین 12000 رکورد برای AssetID=8312 ...")
        query_main = """
        SELECT TOP 12000 [Value], [RecordTime], [RecordDate], [DateTime], [TimeStamp]
        FROM [DSAS].[PDA].[Periodic_Values]
        WHERE [UnitID] = 12 AND [AssetID] = 8312
        ORDER BY [DateTime] DESC
        """
        cursor_sql.execute(query_main)
        rows_main = cursor_sql.fetchall()

        if not rows_main:
            print("هیچ داده‌ای برای AssetID=8312 پیدا نشد.")
            return

        inserted_count = 0

        for idx, row in enumerate(rows_main):
            ts = row.TimeStamp
            if ts in existing_ts:
                print(f"رکورد {idx+1}: TimeStamps={ts} تکراری → رد شد.")
                continue

            dt = row.DateTime
            print(f"\n--- درج رکورد جدید {idx+1} | TimeStamps={ts} ---")

            # ۱. درج رکورد اصلی (فقط AssetID_8312)
            insert_sql = """
            INSERT INTO main_table_mhi_main_trans_12
            (unitID, AssetID_8312, RecordTime, RecordDate, DateTime, TimeStamps)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            try:
                with conn_mysql.cursor() as c:
                    c.execute(insert_sql, (
                        12,
                        float(row.Value) if row.Value is not None else None,
                        dt.strftime('%H:%M:%S'),
                        dt.strftime('%Y-%m-%d'),
                        dt,
                        ts
                    ))
                conn_mysql.commit()
                print("رکورد اصلی درج شد.")
                inserted_count += 1
            except Exception as e:
                print(f"خطا در درج رکورد اصلی: {e}")
                conn_mysql.rollback()
                continue

            # ۲. پر کردن دو فیلد دیگر (8313 و 8314)
            for aid in [8313, 8314]:
                field_name = field_map[aid]

                # ابتدا جستجوی نزدیک (کمتر از ۱۰۰۰ واحد اختلاف TimeStamp)
                near_sql = f"""
                SELECT TOP 1 [Value] 
                FROM [DSAS].[PDA].[Periodic_Values]
                WHERE [UnitID]=12 AND [AssetID]=? AND ABS([TimeStamp] - ?) <= 1000
                ORDER BY ABS([TimeStamp] - ?)
                """
                cursor_sql.execute(near_sql, (aid, ts, ts))
                near_row = cursor_sql.fetchone()

                if near_row and near_row.Value is not None:
                    value = float(near_row.Value)
                    source = "نزدیک"
                else:
                    # اگر نزدیک پیدا نشد → میانگین ۱۰ تای آخر
                    avg_sql = f"""
                    SELECT AVG(CAST([Value] AS FLOAT)) 
                    FROM (
                        SELECT TOP 10 [Value] 
                        FROM [DSAS].[PDA].[Periodic_Values]
                        WHERE [UnitID]=12 AND [AssetID]=?
                        ORDER BY [DateTime] DESC
                    ) sub
                    """
                    cursor_sql.execute(avg_sql, (aid,))
                    avg_row = cursor_sql.fetchone()
                    value = round(float(avg_row[0]), 4) if avg_row and avg_row[0] is not None else None
                    source = "میانگین ۱۰ تایی"

                if value is not None:
                    update_sql = f"UPDATE main_table_mhi_main_trans_12 SET {field_name} = %s WHERE TimeStamps = %s"
                    try:
                        with conn_mysql.cursor() as c:
                            c.execute(update_sql, (value, ts))
                        conn_mysql.commit()
                        print(f"  {field_name} = {value} ({source})")
                    except Exception as e:
                        print(f"  خطا در آپدیت {field_name}: {e}")
                        conn_mysql.rollback()
                else:
                    print(f"  {field_name}: هیچ داده‌ای پیدا نشد.")

        print(f"\nنتیجه این اجرا: {inserted_count} رکورد جدید درج و آپدیت شد.")

    except Exception as e:
        print(f"خطای کلی در اجرا: {e}")
    finally:
        if cursor_sql: cursor_sql.close()
        if conn_sql: conn_sql.close()
        if conn_mysql: conn_mysql.close()

# ================== توقف با Ctrl+C ==================
def signal_handler(sig, frame):
    global RUNNING
    print('\nدریافت سیگنال توقف (Ctrl+C). در حال خروج...')
    RUNNING = False
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

# ================== حلقه اجرا (هر ۶۰ دقیقه) ==================
def scheduler():
    while RUNNING:
        run_task()
        print(f"\nمنتظر {INTERVAL_SECONDS//60} دقیقه بعدی...")
        # خوابیدن با چک کردن هر ثانیه برای امکان Ctrl+C
        for _ in range(INTERVAL_SECONDS):
            if not RUNNING:
                break
            time.sleep(1)

# ================== اجرا ==================
if __name__ == "__main__":
    print("اسکریپت انتقال داده برای main_table_mhi_main_trans_12 شروع شد.")
    print("هر ۶۰ دقیقه یکبار اجرا می‌شود. برای توقف Ctrl+C بزنید.")
    scheduler()
    print("اسکریپت با موفقیت متوقف شد.")

اسکریپت انتقال داده برای main_table_mhi_main_trans_12 شروع شد.
هر ۶۰ دقیقه یکبار اجرا می‌شود. برای توقف Ctrl+C بزنید.

اجرای وظیفه: 2025-11-19 17:42:38
جدول هدف: main_table_mhi_main_trans_12
AssetIDهای مورد استفاده: [8312, 8313, 8314]
اتصال به SQL Server...
اتصال به MySQL...
خواندن TimeStampsهای موجود در MySQL...
0 رکورد موجود در جدول MySQL.
خواندن آخرین 12000 رکورد برای AssetID=8312 ...

--- درج رکورد جدید 1 | TimeStamps=1763557092 ---
رکورد اصلی درج شد.
  AssetID_8313 = 60.0 (نزدیک)
  AssetID_8314 = 5.0 (نزدیک)

--- درج رکورد جدید 2 | TimeStamps=1763529291 ---
رکورد اصلی درج شد.
  AssetID_8313 = 60.0 (نزدیک)
  AssetID_8314 = 5.0 (نزدیک)

--- درج رکورد جدید 3 | TimeStamps=1763497984 ---
رکورد اصلی درج شد.
  AssetID_8313 = 60.0 (نزدیک)
  AssetID_8314 = 5.0 (نزدیک)

--- درج رکورد جدید 4 | TimeStamps=1763473481 ---
رکورد اصلی درج شد.
  AssetID_8313 = 60.0 (نزدیک)
  AssetID_8314 = 5.0 (نزدیک)

--- درج رکورد جدید 5 | TimeStamps=1763443093 ---
رکورد اصلی درج شد.
  AssetID_8313 = 59.0 (نزدیک